# Notebook: Load, Clean, and Analyze Campus Incident Data
This notebook performs data loading and cleaning (Notebook 2) and analysis + visualization (Notebook 3) for the campus incidents dataset. It also displays the corrected Jewish-holiday metrics and matched raw rows produced by the pipeline.

In [ ]:
# Cell 1: imports and helper functions
import os
import re
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Nice defaults
sns.set(style='whitegrid', context='notebook')
pd.set_option('display.max_columns', 200)

# Paths
ROOT = Path('..') if Path('.').resolve().name != 'AntisemitismTrends' else Path('.')
DATA_DIR = Path('../data').resolve() if (Path('../data').exists()) else Path('data')
OUT_DIR = Path('../outputs').resolve() if (Path('../outputs').exists()) else Path('outputs')
OUT_DIR.mkdir(parents=True, exist_ok=True)

RAW_CSV = DATA_DIR / 'campus_reports.csv'
CLEAN_PARQUET = OUT_DIR / 'campus_reports.cleaned.parquet'
METRICS_CSV = OUT_DIR / 'protest_jewish_holiday_metrics_corrected.csv'
MATCHED_RAW = OUT_DIR / 'inspection_jh_raw_matches.csv'
PR_PLOT = OUT_DIR / 'pr_curve_best_holiday_model.png'
RATE_PLOT = OUT_DIR / 'protest_rate_by_jewish_holiday_corrected.png'

print('Paths set:', RAW_CSV, CLEAN_PARQUET, METRICS_CSV)

## Notebook 2 — Load CSV and quick inspection
Read the CSV with safe options, detect malformed lines (comment prefix `//` accepted), and show initial diagnostics.

In [ ]:
# 2.1 Load with protective options
read_opts = dict(
    sep=',',
    quotechar='"',
    encoding='utf-8',
    dtype=str,
    keep_default_na=True,
    na_values=['', 'NA', 'N/A'],
    engine='python'  # more tolerant for tricky rows
)

# Some files have '//' comment sections in exported excerpts; support comment prefix
with open(RAW_CSV, 'r', encoding='utf-8') as f:
    sample = ''.join([next(f) for _ in range(10)])
print('Sample header lines:\n', sample)

raw = pd.read_csv(RAW_CSV, **read_opts, comment='//')
print('Loaded raw shape:', raw.shape)
print(raw.dtypes)
raw.head(5)

## Notebook 2 — Parse dates and enforce dtypes
Parse `Date of Incident` to datetime, show parsing failures, and cast categorical columns.

In [ ]:
# 2.2 Parse dates
date_col = 'Date of Incident'
raw[date_col+'_parsed'] = pd.to_datetime(raw[date_col], format='%m/%d/%y', errors='coerce')
# fallback: try infer for the rows that failed
mask_fail = raw[date_col+'_parsed'].isna()
if mask_fail.any():
    raw.loc[mask_fail, date_col+'_parsed'] = pd.to_datetime(raw.loc[mask_fail, date_col], errors='coerce', dayfirst=False)

print('Parsed datetimes: total failures =', raw[date_col+'_parsed'].isna().sum())
raw.loc[raw[date_col+'_parsed'].isna(), [date_col]].head(10)

# Enforce categorical dtypes
cat_cols = ['State of Incident', 'Incident Type', 'College/University']
for c in cat_cols:
    if c in raw.columns:
        raw[c] = raw[c].astype('category')

raw[date_col+'_parsed'].head()

## Notebook 2 — Clean Description text (quotes, escape sequences, PII)
Normalize the `Description` field, mask phone numbers and emails, compute description length and flags.

In [ ]:
# 2.3 Clean Description
desc_col = 'Description'
if desc_col in raw.columns:
    def mask_pii(text):
        if pd.isna(text):
            return text
        t = str(text)
        # unescape doubled quotes
        t = t.replace('""', '"')
        # mask phone numbers
        t = re.sub(r'\b\d{3}[-.\s]?\d{3}[-.\s]?\d{4}\b', '[PHONE]', t)
        # mask emails
        t = re.sub(r'[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}', '[EMAIL]', t)
        return t.strip()

    raw['description_clean'] = raw[desc_col].apply(mask_pii)
    raw['description_len'] = raw['description_clean'].fillna('').str.len()
    raw['description_empty'] = raw['description_len'] == 0
    display(raw[[desc_col, 'description_clean', 'description_len', 'description_empty']].head(5))
else:
    print('No Description column found')

## Notebook 2 — Normalize College/University names and locations
Apply a small canonical mapping; fallback to original. Validate states against the USPS state list.

In [ ]:
# 2.4 Normalize College names (small mapping example)
from hashlib import md5

college_col = 'College/University'
if college_col in raw.columns:
    mapping = {
        'California State University, Northridge': 'CSUN',
        'San Francisco State University': 'San Francisco State',
        # Add more mappings as needed
    }
    raw['college_canonical'] = raw[college_col].map(mapping).fillna(raw[college_col])
    # create campus_id as hash of canonical name + city
    raw['campus_id'] = raw[['college_canonical', 'City of Incident']].fillna('').apply(lambda row: md5((str(row['college_canonical']) + '|' + str(row['City of Incident'])).encode('utf-8')).hexdigest(), axis=1)
    raw[['College/University', 'college_canonical', 'campus_id']].head()
else:
    print('No College/University column found')

# Validate State of Incident against USPS two-letter list
usps = set(['Alabama','Alaska','Arizona','Arkansas','California','Colorado','Connecticut','Delaware','Florida','Georgia','Hawaii','Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine','Maryland','Massachusetts','Michigan','Minnesota','Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire','New Jersey','New Mexico','New York','North Carolina','North Dakota','Ohio','Oklahoma','Oregon','Pennsylvania','Rhode Island','South Carolina','South Dakota','Tennessee','Texas','Utah','Vermont','Virginia','Washington','West Virginia','Wisconsin','Wyoming','Washington D.C.'])
if 'State of Incident' in raw.columns:
    raw['state_valid'] = raw['State of Incident'].isin(usps)
    print('Invalid state count:', (~raw['state_valid']).sum())
    raw.loc[~raw['state_valid'], 'State of Incident'].value_counts().head(10)

## Notebook 2 — Handle missing values and duplicates
Identify missing values and deduplicate rows; add `is_duplicate` flag and drop duplicates.

In [ ]:
# 2.5 Missing values summary
missing = raw.isna().sum().sort_values(ascending=False)
print('Missing per column:\n', missing[missing>0].head(20))

# De-duplicate: consider date + campus_id + incident type + description hash
raw['desc_hash'] = raw['description_clean'].fillna('').apply(lambda x: md5(x.encode('utf-8')).hexdigest())
subset = [date_col+'_parsed', 'campus_id', 'Incident Type', 'desc_hash']
raw['is_duplicate'] = raw.duplicated(subset=subset, keep='first')
print('Duplicates found:', raw['is_duplicate'].sum())

# drop duplicates for cleaned dataset
clean = raw[~raw['is_duplicate']].copy()
print('Cleaned shape after dropping duplicates:', clean.shape)

## Notebook 2 — Validate and save cleaned CSV / parquet
Run schema checks and save cleaned data to compressed parquet and CSV.

In [ ]:
# 2.6 Basic schema checks
required = [date_col+'_parsed', 'campus_id', 'Incident Type']
for r in required:
    if r not in clean.columns:
        raise RuntimeError(f'Missing required column: {r}')
    if clean[r].isna().any():
        print(f'Warning: nulls in required column {r}:', clean[r].isna().sum())

# Save: try parquet, fall back to CSV if parquet engine missing
csv_path = OUT_DIR / 'campus_reports.cleaned.csv'
try:
    import pyarrow  # type: ignore
    clean.to_parquet(CLEAN_PARQUET, index=False, compression='brotli')
    print('Saved cleaned data to', CLEAN_PARQUET)
    # quick reload assert
    _back = pd.read_parquet(CLEAN_PARQUET)
    assert len(_back) == len(clean)
    print('Roundtrip rows OK (parquet)')
except Exception as e:
    # fallback
    clean.to_csv(csv_path, index=False)
    print('Parquet unavailable; saved cleaned CSV to', csv_path)
    _back = pd.read_csv(csv_path)
    assert len(_back) == len(clean)
    print('Roundtrip rows OK (csv)')


---
# Notebook 3 — Analysis & Visualization
Load cleaned data and prepare aggregated tables, time-series, maps, and NLP summaries.

In [ ]:
# 3.1 Load cleaned data
try:
    clean = pd.read_parquet(CLEAN_PARQUET)
    print('Loaded cleaned shape (parquet):', clean.shape)
except Exception:
    csv_path = OUT_DIR / 'campus_reports.cleaned.csv'
    if csv_path.exists():
        clean = pd.read_csv(csv_path)
        print('Loaded cleaned shape (csv):', clean.shape)
    else:
        raise RuntimeError('Cleaned data not found: tried parquet and csv')

clean.head(3)

## Notebook 3 — Aggregate incidents by date, state, and campus
Compute daily, weekly, and monthly counts; save aggregated DataFrames.

In [ ]:
# 3.2 Aggregations
# Ensure the parsed date column is a datetime64 dtype so pd.Grouper works
clean[date_col+'_parsed'] = pd.to_datetime(clean[date_col+'_parsed'], errors='coerce')
# keep a simple python date column for daily counts/display
clean['date'] = clean[date_col+'_parsed'].dt.date

daily = clean.groupby('date').size().rename('incidents').reset_index()
# weekly/monthly using the datetime column (requires datetime64 dtype)
weekly = clean.groupby(pd.Grouper(key=date_col+'_parsed', freq='W'))['campus_id'].count().rename('incidents').reset_index()
monthly = clean.groupby(pd.Grouper(key=date_col+'_parsed', freq='M'))['campus_id'].count().rename('incidents').reset_index()

daily.head(), weekly.head(), monthly.head()

# save
daily.to_csv(OUT_DIR / 'agg_daily.csv', index=False)
weekly.to_csv(OUT_DIR / 'agg_weekly.csv', index=False)
monthly.to_csv(OUT_DIR / 'agg_monthly.csv', index=False)
print('Saved aggregated tables')

## Notebook 3 — Time-series plots and rolling averages
Plot daily counts with 7-day and 30-day rolling means, annotate notable dates.

In [ ]:
# 3.3 Time-series
daily['date'] = pd.to_datetime(daily['date'])
daily = daily.sort_values('date')
daily['rolling7'] = daily['incidents'].rolling(7, center=True, min_periods=1).mean()
daily['rolling30'] = daily['incidents'].rolling(30, center=True, min_periods=1).mean()

plt.figure(figsize=(14,5))
plt.plot(daily['date'], daily['incidents'], alpha=0.5, label='Daily')
plt.plot(daily['date'], daily['rolling7'], label='7-day MA')
plt.plot(daily['date'], daily['rolling30'], label='30-day MA')
plt.legend()
plt.title('Campus incidents: daily counts and rolling means')
plt.tight_layout()
plot_path = OUT_DIR / 'daily_time_series.png'
plt.savefig(plot_path)
print('Saved', plot_path)
plt.show()

## Notebook 3 — Incident-type breakdowns and top campuses
Show counts by incident type and top campuses.

In [ ]:
# 3.4 Incident type breakdown
types = clean['Incident Type'].value_counts().reset_index()
types.columns = ['Incident Type', 'count']
plt.figure(figsize=(8,5))
sns.barplot(data=types, x='count', y='Incident Type')
plt.title('Incidents by type')
plt.tight_layout()
plt.savefig(OUT_DIR / 'incidents_by_type.png')
print('Saved incidents_by_type.png')
plt.show()

# Top campuses
top_campuses = clean['college_canonical'].value_counts().head(20).reset_index()
top_campuses.columns = ['college', 'count']
top_campuses.head()

## Notebook 3 — Basic NLP on Description
Compute top tokens and bigrams (basic). Wordcloud optional.

In [ ]:
# 3.5 Basic NLP (token counts)
from collections import Counter
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))

text_series = clean['description_clean'].dropna().astype(str)
all_text = ' '.join(text_series)
# simple tokenization
tokens = [t.lower() for t in re.findall(r"\b\w{3,}\b", all_text)]
filtered = [t for t in tokens if t not in stop]
most = Counter(filtered).most_common(30)
most[:20]

# Save top tokens
pd.DataFrame(most, columns=['token','count']).to_csv(OUT_DIR / 'top_tokens.csv', index=False)
print('Saved top_tokens.csv')

## Notebook 3 — Export figures and summary tables
List saved artifacts and show corrected Jewish-holiday metrics + matched raw rows (if present).

In [ ]:
# 3.6 Show saved artifacts and the corrected metrics + matched rows
artifacts = sorted([str(p) for p in OUT_DIR.glob('*')])
print('Artifacts in outputs:')
for a in artifacts:
    print(' -', a)

# Display corrected metrics CSV if present
if METRICS_CSV.exists():
    print('\nCorrected Jewish-holiday metrics:')
    display(pd.read_csv(METRICS_CSV))
else:
    print('\nNo corrected metrics CSV found at', METRICS_CSV)

# Display matched raw rows
if MATCHED_RAW.exists():
    print('\nMatched raw rows (first 30):')
    matched = pd.read_csv(MATCHED_RAW)
    display(matched.head(30))
else:
    print('\nNo matched raw rows file found at', MATCHED_RAW)

# Display relevant plots if present
from IPython.display import Image, display
for p in [RATE_PLOT, PR_PLOT, OUT_DIR / 'daily_time_series.png']:
    if Path(p).exists():
        print('Showing:', p)
        display(Image(str(p)))
    else:
        print('Missing plot:', p)

## Short Summary — Jewish-holiday vs protest check (from pipeline)
- Contingency and Fisher exact test were computed in the pipeline and saved to `outputs/protest_jewish_holiday_metrics_corrected.csv`.
- The pipeline-run result (using the user-provided canonical holiday ranges) yielded roughly similar protest rates on Jewish holidays vs non-holidays; see the metrics table above for exact counts and p-value.